# Image Filtering

## Convolution / Cross-Correlation

A Jupyter notebook showing the behaviour of various 2-d smoothing filters.

FIrst some setup, load the required libraries and an image.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


In [ ]:
im = np.array(Image.open("fruits.jpg").convert('L')) / np.float32(255.0)   # Normalised 0...1 image.

In [ ]:
print(f"The image is {im.shape[0]} rows by {im.shape[1]} columns, with datatype {im.dtype}")
print(f"Its min value is {im.min()} and its max is {im.max()}")

In [ ]:
plt.figure('Fruits Image', figsize=(6,6))
plt.imshow(im, cmap='gray', vmin=0, vmax=1);  # Stop auto-scaling of brightness.


The basic operation in all forms of image filtering is 2-D convolution (or, equivalently, 2-D cross-correlation).  

In 2-D convolution the basic equation is:

\begin{equation}
                Y[r,c] = \sum_{u=-n}^{n} \sum_{v=-m}^{m} H[u,v] \; X[r-u,c-v] 
\end{equation}


Here the input image is $X$, the output image is $Y$ and the convolution kernel is represented by $H$, and is assumed to be $(2n+1)\times(2m+1)$.  This,  of course, assumes that the indices of the kernel can be negative, and ignores what happens at the edges of the image.

In Python, as with most other languages, negative array indices are not allowed, so we can rewrite the equation to eliminate them (substituting $i = u + n$ and $j = v + m$ into the indices for $X$):

\begin{eqnarray*}
               Y[r,c] & = & \sum_{i=0}^{2n+1} \sum_{j=0}^{2m+1} H[i,j] \; X[r-(i-n),c-(j-n)]\\
                      & = & \sum_{i=0}^{2n+1} \sum_{j=0}^{2m+1} H[i,j] \; X[r+n-i,c+m-j]
\end{eqnarray*}

The range of indices for the kernel $H$ now runs over $ i = 0 \ldots 2n+1$, $j = 0 \ldots 2m+1$.
It is just necessary to ensure that the indices to $X$ never transgress the array bounds, and this equation can be implemented directly in interpreted Python.


In [ ]:
def convolution_v1(X,H):
    """Naive, direct implementation of convolution.  Note edge effect: (n,m) border not filtered."""
    HR, HC = H.shape
    assert HR % 2 == 1 and HC % 2 == 1, "Odd-sized kernel required."
    n, m = HR // 2, HC // 2
    R, C = X.shape
    Y = np.zeros(X.shape, X.dtype)
    for r in range(n,R-n):
        for c in range(m,C-m):
            acc = 0.0
            for i in range(HR):
                for j in range(HC):
                    acc += H[i,j] * X[r + n - i,c + m - j]
            Y[r,c] = acc
    return Y

Create a simple test image to examine the effects of convolution filtering.

In [ ]:
X = np.zeros((100,100),dtype='float32')
X[30:70,30:70] = 1.0
plt.figure()
plt.imshow(X, cmap='gray');

Here we try out the convolution on various $N \times N$ constant kernels.  Note that $N$ must be odd, and that the integral of the kernel (i.e., its sum) should be 1, so that the average brightness of the output image isn't changed relative to the input.

In [ ]:
def makebox(k):
    "Create a (2k+1) x (2k+1) box filter kernel suitable for convolution filtering."
    w = 2 * k + 1
    return np.ones((w,w),dtype='float32') / w**2

filter_ks = [1,2,5]
Hs = [makebox(k) for k in filter_ks]

In [ ]:
Xouts = [convolution_v1(X,H) for H in Hs]

In [ ]:
plt.figure('Box filtering', figsize=(12,4))
for i in range(len(Xouts)):
    plt.subplot(1,3,(i+1))
    plt.imshow(Xouts[i], cmap='gray')
    w = 2 * filter_ks[i] + 1
    plt.title("{} x {}".format(w,w)) ; plt.axis('off')
plt.show();

The filter does not change the brightness of the image.

In [ ]:
def avgbright(im):
    "Return the averge brightness of an image."
    return np.sum(im.ravel())/(im.shape[0]*im.shape[1])
    
print("Average brightness, original image and filtered versions")
print([avgbright(Xi) for Xi in (X, Xouts[0], Xouts[1], Xouts[2])])

Here are various box filters applied to the fruits image.

In [ ]:
H3, H5, H11 = Hs   # Get kernels under convenient names.

In [ ]:
## 3 x 3 convolution using interpreted Python.  Quite slow for a moderately-sized image.

im1 = convolution_v1(im,H3)

In [ ]:
## No real visible difference between this and the original.

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(im1, cmap=plt.cm.gray, vmin=0, vmax=1)
plt.title("Fruits image: 3x3 box filter")
plt.subplot(1,2,2)
plt.imshow(im, cmap=plt.cm.gray, vmin=0, vmax=1)
plt.title("Original fruits image");


In [ ]:
## 5 x 5 convolution using interpreted Python.  Slower.

im2 = convolution_v1(im,H5)

In [ ]:
## Some slight visible differences appearing.

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(im2, cmap=plt.cm.gray, vmin=0, vmax=1)
plt.title("Fruits image: 5x5 box filter")
plt.subplot(1,2,2)
plt.imshow(im, cmap=plt.cm.gray, vmin=0, vmax=1)
plt.title("Original fruits image");


In [ ]:
## 11 x 11 convolution using interpreted Python.  Very slow.

im3 = convolution_v1(im,H11)

In [ ]:
## This time, quite distinct blurring.

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(im3, cmap=plt.cm.gray, vmin=0, vmax=1)
plt.title("Fruits image: 11x11 box filter")
plt.subplot(1,2,2)
plt.imshow(im, cmap=plt.cm.gray, vmin=0, vmax=1)
plt.title("Original fruits image")
plt.show();

## Convolution using Numba

Convolution using interpreted Python is, unfortuantely, very slow.  Convolution is an expensive operation, with an $R\times C$ source image and a $N \times M$ kernel, the costs rise as $O(RCNM)$.  Clearly, interpreted Python isn't going to perform well.  On my 2.7GHz MacBook (admittedly an old machine), the $5 \times 5$ kernel takes about $6$ seconds to convolve a $480 \times 512$ image.  The $11 \times 11$ kernel takes about $23$ seconds - $3.75$ times as long (note that $(11\times 11)/(5\times 5) = 4.84$).  You may get different numbers (the Anaconda implmentation on Windows seems to be faster), but the slowdown will be approximately the same.  The slowdown is sublinear because the inner for-loops have wider "spans" in the larger kernel, so the loop setup/teardown overhead is smaller as a proportion of overall run-time.

In [ ]:
timeh5 = %timeit -o convolution_v1(im,H5)

In [ ]:
timeh11 = %timeit -o convolution_v1(im,H11)

In [ ]:
print(f"Time ratio = {timeh11.average/timeh5.average:.4f}, "
      f"kernel size ratio = {(11*11)/(5*5)}.")

Numba is an optimising compiler for a subset of Python.  It can't compile the whole Python language, but is very good at compiling regular code involving loops and Numpy arrays (this is what it is designed for).

One obvious approach to speeding up the filters is to recode the convolution using Numba, this should give a dramatic speed increase as we move from interpreted to compiled code, but with very minimal changes to the original code.

In [ ]:
import numba
from numba import njit

Here is a Numba implementation of a convolution filter.  It is as simple as prepreding the original convolution code with the <font color="#a000a0"><tt>@njit</tt></font> Numba "decorator", saying, in effect, "compile this function to machine code".  Note that the input image is assumed to be single-channel.

In [ ]:
@njit
def convolution_v2(X,H):
    """Numba implementation of convolution.  Note edge effect: (n,m) border not filtered."""
    HR, HC = H.shape
    assert HR % 2 == 1 and HC % 2 == 1, "Odd-sized kernel required."
    n, m = HR // 2, HC // 2
    R, C = X.shape
    Y = np.zeros(X.shape, X.dtype)
    for r in range(n,R-n):
        for c in range(m,C-m):
            acc = 0.0
            for i in range(HR):
                for j in range(HC):
                    acc += H[i,j] * X[r + n - i,c + m - j]
            Y[r,c] = acc
    return Y


In [ ]:
im4 = convolution_v2(im, H5)    # Compiled Numba convolution using the 5 x 5 box kernel.

In [ ]:
plt.figure('Fruits image, 5 x 5 convolution in Numba', figsize=(6,6))
plt.imshow(im4, cmap='gray', vmin=0, vmax=1);


In [ ]:
im5 = convolution_v2(im, H11)   # Compiled Numba convolution using the 11 x 11 box kernel.

In [ ]:
plt.figure('Fruits image, 11 x 11 convolution using Numba', figsize=(6,6))
plt.imshow(im5, cmap='gray', vmin=0, vmax=1);


In [ ]:
np.allclose(im2,im4)  # Are the results of the Numba convolution to same as those of the plain Python?

In [ ]:
np.allclose(im3,im5) # Comparing 11x11 filtering results.

Let's benchmark the new Numba code.

In [ ]:
timeh5n = %timeit -o convolution_v2(im,H5)

In [ ]:
timeh5.average/timeh5n.average   # Compiled vs interpreted for a 5 x 5 box filter

In [ ]:
timeh11n = %timeit -o convolution_v2(im,H11)

Comparing the compiled to the interpreted convolution on the $11 \times 11$ box kernel.  The dramtic speedup still applies.  This is for a Macbook Pro running Anaconda Python 3.7.9.  Times may vary for other implementations...

In [ ]:
timeh11.average / timeh11n.average

In [ ]:
H51 = makebox(25)   # 51 x 51 box filter

In [ ]:
im6 = convolution_v2(im, H51)   # Numba compiled using a 51 x 51 box.  Reasonably fast.

In [ ]:
plt.figure('Fruits image, 51 x 51 convolution using Numba', figsize=(6,6))
plt.imshow(im6, cmap='gray');   # Autoscale to full range, this makes the image appear brighter than it really is.


In [ ]:
timeh51n = %timeit -o convolution_v2(im,H51)

In [ ]:
51**2/11**2   # Ratio of kernel sizes for 51 x 51 and 11 x 11 box kernels.

In [ ]:
timeh51n.average / timeh11n.average      # Ratio of run times.  Roughly the same increase.

As you can see, there is a dramatic performance improvement, the compiled code is at least orders of magnitude faster than the interpreted code.  However, the $O(RCNM)$ scaling still applies, so bigger kernals are still quadratically slower than smaller ones even with Numpy assistance.

Of course, the edges are not being filtered.  We can address this by assuming that the image is embedded in a "sea of zeros", so that indices going outside the image bounds contribute nothing to the final filtered image.  This allows filtering up the the image edges, but with a perceptable darkening.

Another point to note is that something strange is happening at the edges of the filtered image.  Because our convolution assumes that the source image is bounded by zeros, a noticable "darkening" or discontinuity happens at the image edges, especially for big kernels like the $51 \times 51$ example.

One way to avoid this is to change the boundary conditions of the source image.  Instead of assuming that it is embedded in a "sea of zeros", let's assume that it is surrounded by "reflections" of itself.  




In [ ]:
@njit
def convolution_v3(X, H):
    """Numba-compiled 2-d convolution of image *X* with kernel *H*, extending out
       to image edges."""
    HR, HC = H.shape
    assert HR % 2 == 1 and HC % 2 == 1, "Odd-sized kernel required."
    n, m = HR // 2, HC // 2
    R, C = X.shape
    Y = np.zeros(X.shape, X.dtype)
    for r in range(R):
        for c in range(C):
            acc = 0.0
            for i in range(HR):
                u = r + n - i
                if u >= 0 and u < R:
                    for j in range(HC):
                        v = c + m - j
                        if v >= 0 and v < C:
                            acc += H[i,j] * X[u,v]
            Y[r,c] = acc
    return Y


In [ ]:
im7 = convolution_v3(im,H51)

In [ ]:
plt.figure('Fruits image, 51 x 51 convolution using Numba with edge processing.', figsize=(6,6))
plt.imshow(im7, cmap='gray');     # Autoscale.


Now the edges are being filtered, but they are noticably darker than the image body.  This is because the wide ($51\times 51$) convolution filter is assuming that anything beyound the edges of the image is $0$.  So consider $Y[0,0]$, it's going to get a lot of zeros, hence appear dim.

One way to avoid this is to change the boundary conditions of the source image.  Instead of assuming that it is embedded in a "sea of zeros", let's assume that it is surrounded by "reflections" of itself.  


Note one other thing going on in this code.  We pass the "parallel=True" flag to Numba and tell it that the outer loop ($r$ index) may be parallelize if possible.  This can speed up things by distributing the computation across multiple cores.  I find, for this code, a speedup using parallel of about 2 to 3 times over the non-parallel Numba version.  Numba is really an excellent system, allowing algorithms to be written and tested in simple Python, but then allowing fast run times through a clever, type-inferencing, JIT compiler combined with the ability to parallelize code, all with minimal effort on the part of the programmer.


In [ ]:
@njit(parallel=True)
def convolution_v4(X, H):
    """Numba-compiled 2-d reflection convolution of image *X* with kernel *H*, 
       extending out to image edges."""
    HR, HC = H.shape
    assert HR % 2 == 1 and HC % 2 == 1, "Odd-sized kernel required."
    n, m = HR // 2, HC // 2
    R, C = X.shape
    Y = np.zeros(X.shape, X.dtype)
    for r in numba.prange(R):                 # Tell Numba that this loop may be parallelized.
        for c in range(C):
            acc = 0.0
            for i in range(HR):
                u = r + n - i
                if u < 0: u = -u              # If u beyound left margin, reflect it back,
                elif u >= R: u = 2*R-u-1      # and ditto if beyond right margin.
                for j in range(HC):
                    v = c + m - j
                    if v < 0: v = -v          # If v above top of image, reflect it back,
                    elif v >= C: v = 2*C-v-1  # and ditto if below bottom.
                    acc += H[i,j] * X[u,v]
            Y[r,c] = acc
    return Y


In [ ]:
im6 = convolution_v4(im,H51)

In [ ]:
plt.figure('Fruits image, 51 x 51 (reflection) convolution in Numba', figsize=(6,6))
plt.imshow(im6, cmap='gray');


In [ ]:
%timeit convolution_v3(im,H51)     #  This is non-parallel Numba-compiled code.

In [ ]:
%timeit convolution_v4(im,H51)     #  And this is Numba code with parallel=True.

Here we can see the effects of the original zero padding and the new "reflection" padding on the top-left corner of the image.

In [ ]:
plt.figure()
plt.subplot(121)
plt.imshow(convolution_v3(im,H11)[:50,:50],cmap='gray',vmin=0,vmax=1)
plt.title('zero padding')
plt.subplot(122)
plt.imshow(convolution_v4(im,H11)[:50,:50],cmap='gray',vmin=0,vmax=1)
plt.title('reflection');

Below we have a definiiton for "circular convolution".  This assumes that the image and the convolution kernel are both periodic.  It is important, because this is the assumption made implicitly by the Fourier transform.

In [ ]:
@njit(parallel=True)
def convolution_v5(X, H):
    """Numba-compiled 2-d convolution of image *X* with kernel *H*, extending out
       to image edges."""
    HR, HC = H.shape
    assert HR % 2 == 1 and HC % 2 == 1, "Odd-sized kernel required."
    n, m = HR // 2, HC // 2
    R, C = X.shape
    Y = np.zeros(X.shape, X.dtype)
    for r in numba.prange(R):                 # Tell Numba to parallelize the outer loop.
        for c in range(C):
            acc = 0.0
            for i in range(HR):
                u = r + n - i
                if u < 0: u += R              # If u beyound left margin, take from right side of image.
                elif u >= R: u -= R           # If beyond right margin take from left side.
                for j in range(HC):
                    v = c + m - j
                    if v < 0: v += C          # If v above top of image, take from bottom.
                    elif v >= C: v -= C       # If v below bottom, take from top.
                    acc += H[i,j] * X[u,v]
            Y[r,c] = acc
    return Y


In [ ]:
plt.figure('Fruits image, 51 x 51 circular convolution in Numba', figsize=(6,6))
plt.imshow(convolution_v5(im,H51), cmap='gray');

## Filtering using scipy.ndimage.filters

The library <font color="#2020ff"><b>scipy.ndimage.filters</b></font> contains a set of predefined, and very efficiently implemented, routines for image filtering using both predefined and user-defined kernels.

<p>
The most important routine from <font color="#2020ff"><b>scipy.ndimage.filters</b></font> is <font color="#2020ff"><b>gaussian_filter</b></font>.  This implements an $N$-d Gaussian filter.  The $2$-d version implements the kernel:

$$
H(x,y) = {1 \over 2 \pi\sigma^2} \exp\left( - {x^2 + y^2} \over {2\sigma^2} \right)
$$

where $\sigma$ is the standard deviation of the kernel (in pixel units).

In [ ]:
import scipy.ndimage.filters as filters

In [ ]:
plt.figure('Gaussian Filtering', figsize=(12,10))
sigmas = [2.0, 5.0, 10.0, 25.0]
for i in range(len(sigmas)):
    plt.subplot(2, int(np.ceil(len(sigmas)//2)), (i+1))
    plt.imshow(filters.gaussian_filter(im,sigmas[i]), cmap=plt.cm.gray)
    plt.title("S.D. $\sigma$ = {}".format(sigmas[i])) ; plt.axis('off') ;
    

<p>
Other filter types in the library include the nonlinear median, maximum and minimum filters.

<p>
Here is a typical use of a $3 \times 3$ median filter for the removal of salt and pepper noise. The bridge image is $512 \times 512 \times 8$bits, and has been corrupted by $0.25\%$ black $(0)$ pixels and $0.25\%$ white $(255)$ ones.  A $3 \times 3$ median filter is largely successful in removing this noise without blurring the underlying image too much.


In [ ]:
noisy_bridge = np.array(Image.open('bridge-sp-noise-005.png').convert('L'))

In [ ]:
bridge = filters.median_filter(noisy_bridge,3)

In [ ]:
plt.figure("Median Filtering",figsize=(14,8))
for i,im,title in zip([1,2], [noisy_bridge, bridge], ["Original", "3x3 Median"]):
    plt.subplot(1,2,i) 
    plt.imshow(im, cmap=plt.cm.gray) ; plt.axis('off')
    plt.title(title);

<p>
Other useful rank-order filters are the minimum and maximum.  When used with binary images, these have special functions:
<ol>
<li> The maximum filter provides the <em>binary dilation</em> operation, which tends to "fill in" gaps in binary images.
<li> The minimum filter provides the <em>binary erosion</em> operation, which tends to thin down thick lines.
</ol>

<p>
Here binary dilation (a maximum filter) is first used to fill-in gaps in a binary image of a circle, then binary erosion (a minimum filter) thins down the shape.

<p>
Note the masks used for these filters.  The maximum filter uses a $3 \times 3$ square mask, so the centre pixel is set in the output if <em>any</em> of the pixels in the input covered by the mask are set.  Such a mask is known as an "8-neighbourhood".  The minimum filter, on the other hand, uses a "cross-shaped" mask, known as a "4-neighbourhood". In this case, the output pixel at $(r,c)$ is only set in the output if <em>all</em> pixels at $(r,c)$, $(r-1,c)$, $(r+1,c)$, $(r,c-1)$ and $(r,c+1)$ are set in the input.


In [ ]:
broken_circ = np.array(Image.open("broken-circle.png").convert('L'))

In [ ]:
dilated_circ = filters.maximum_filter(broken_circ,3)

In [ ]:
eroded_circ = filters.minimum_filter(dilated_circ,footprint=[[0,1,0],[1,1,1],[0,1,0]])

In [ ]:
plt.figure("Binary Dilation and Erosion", figsize=(16,8))
for i,im,title in zip([1,2,3], [broken_circ, dilated_circ, eroded_circ], 
                      ["Original", "Dilation", "Erosion"]):
    plt.subplot(1,3,i) 
    plt.imshow(im, cmap=plt.cm.gray, interpolation='nearest') ; plt.axis('off') 
    plt.title(title)

### Image Sharpening

We can sharpen an image by differencing an image and its smoothed version to generate edge regions, then add the difference back to the original.  The smoothing filter can be a box, or a Gaussian, the latter generally perfoming better, and tending to approximate a "Laplacian of Gaussian" operation, $\Delta G = \nabla^2 G = \frac{\partial^2 G}{\partial x^2} + \frac{\partial^2 G}{\partial y^2}$.

Note that, in this section, we use the scipy.ndimage.filters implementation of the box filter, "uniform_filter".  Generally it is better to use the routines from scipy rather than "roll your own".

In [ ]:
barbara = np.array(Image.open('barbara.png').convert('L'))

In [ ]:
barbara = (barbara - barbara.min())/float(barbara.max() - barbara.min()) # 0..1 float image

In [ ]:
b_detail = barbara - filters.uniform_filter(barbara,5) # 5 x 5 box filter

In [ ]:
b_sharp = np.clip(barbara + b_detail, 0, 1)

In [ ]:
plt.figure('Sharpening by Box Filter', figsize=(14,14))
for i,im,title in zip([1,2,3], [barbara, b_detail, b_sharp], 
                      ["Original", "Detail", "Sharpened (5x5 box)"]):
    plt.subplot(2,2,i) 
    plt.imshow(im, cmap=plt.cm.gray, interpolation='nearest') ; plt.axis('off') 
    plt.title(title);

Gaussian filters typically give better looking results.  Here we have a Gaussian with $\sigma = 3$.

In [ ]:
b_sharp_g3 = np.clip(2*barbara - filters.gaussian_filter(barbara,3), 0, 1)

In [ ]:
plt.figure('Sharpening by Gaussian', figsize=(14,14))
for i,im,title in zip([1,2,3], [barbara, b_sharp, b_sharp_g3], 
                      ["Original", "5x5 box", "Gaussian, $\sigma=3$"]):
    plt.subplot(2,2,i) 
    plt.imshow(im, cmap=plt.cm.gray, interpolation='nearest') ; plt.axis('off')
    plt.title(title)

The sharpening can also be controlled by the parameter $\alpha$, where the final image is generated from $I' = (1+\alpha)I - \alpha(I \ast H)$.

In [ ]:
def sharpen(image, sigma, alpha):
    "Sharpen a normalised (0..1) float image, sigma is the s.d. of a Gaussian."
    return np.clip((1+alpha)*image - alpha*filters.gaussian_filter(image,sigma), 0, 1)

In [ ]:
plt.figure('Varying $\\alpha$', figsize=(14,14))
for i in range(4):
    plt.subplot(2,2,i+1) 
    plt.imshow(sharpen(barbara,3,2**i), cmap=plt.cm.gray, interpolation='nearest') ; plt.axis('off')
    plt.title(f"$\\alpha = {2**i}$, $\\sigma = 3$")

## Convolution by Fourier Transform

One way to try to tackle the cost of doing convolution of images with big kernels (like the $51 \times 51$ kernel shown earlier) is to use the fast Fourier transform.  Convolution in the "time" domain is equivalent to multiplication in the "frequency" domain, so the idea is to take the Fourier transforms of the source image and the convolution kernel, multiply them and take the inverse transform of their product.  Since the fast Fourier transform is efficient, this should be faster for large images.

Numpy provides routines to take discrete Fourier transforms of images.  The basic routines needed are rfft2 (a FFT of a 2-d real source) and irfft2 (an inverse transform back to a 2-d real image from its 2-d frequency-domain representation).


In [ ]:
import numpy.fft as fft
im = np.array(Image.open('fruits.jpg').convert('L')) / np.float32(255)
f_im=fft.rfft2(im)

Note that the transform of a real-valued (not complex) source is a complex array that has just over half the number of array elements.  Note that the entries of the transform are complex numbers, so the amount of information is the same.  

In [ ]:
f_im.shape, f_im.dtype, im.shape, im.dtype

Here are plots showing the log of the magnitude of the image's transform, and the phase information (in degrees).

In [ ]:
plt.figure("Fourier Transform of Image", figsize=(12,6))
for i,the_image,title in zip([1,2],
                             [np.log(np.abs(f_im)+1),     # Note add 1 to allow log to work with 0's.
                              np.angle(f_im, deg=True)],
                             ["Log Magitude", "Phase"]):
    plt.subplot(1,2,i)
    plt.imshow(the_image,cmap='coolwarm',interpolation='none')
    plt.title(title)
    plt.colorbar()
plt.show()

We can also take the transform of the kernel.  Here f_H51 is the discrete Fourier transform of a $51 \times 51$ block smoothing filter.  Note that the transform has only a single component, at "DC", as the filter is a constant.

In [ ]:
f_H51 = fft.rfft2(H51)

In [ ]:
f_H51.shape

Here we plot the magnitude of the kernel and its phase.  Note that this time we don't use the log of the magnitude, because there is only one non-zero component.  Clearly we can't take the log of zero.

The majority of the phase information is meaningless in this case, because it is associated with frequency components of zero magnitude.

In [ ]:
plt.figure("Fourier Transform of Kernel", figsize=(12,6))
for i,the_image,title,color in zip([1,2],
                                   [np.abs(f_H51),
                                    np.angle(f_H51, deg=True)],
                                   ["Magitude", "Phase"],
                                   ['gray', 'coolwarm']):
    plt.subplot(1,2,i)
    plt.imshow(the_image,cmap=color,interpolation='none')
    plt.title(title)
    plt.colorbar()
plt.show()

In [ ]:
print(f_H51.max(), f_H51.min())

One problem that immediately presents itself is that the transform of the image and the transform of the kernel are different sizes, so they can't be multiplied together in frequency space.  One way to overcome this is to pad the smaller input with zeros to make it the same size as
the larger.  Then the transforms of the two sources will be the same size, and can be multiplied.

Here the image, "im", is clearly expected to be larger than the kernel "H", so we first pad the kernel with zeros to make it the same size as the image, take the transforms of the image and the zero-padded kernel, multiply them, and take the inverse transform of the product.  This
gives a first attempt at convolution using the Fourier transform.


In [ ]:
def convolution_fft_v1(im,H):
    """Perform convolution of an image by a kernel using the Fourier transform."""
    dft_im = fft.rfft2(im)
    dft_H = fft.rfft2(H, im.shape)    # Note the second parameter.  This zero-pads H to the shape of im.
    return fft.irfft2(dft_im * dft_H)

We try this on the image "im" and a $51 \times 51$ constant kernel.  It's a bit faster than direct convolution for this large kernel, and the result has similar looking statistics.

In [ ]:
out_f = convolution_fft_v1(im,H51)

However, we we display the results, we see that the fft-based convolution contains significant artifacts.  The output is shifted to the right and down relative to the source and the direct convolution, and there seem to be issues with the edges of the fft convolution.  We see that there appears to be some "wrap around", particularly from the bottom of the image to the top.

In [ ]:
out_direct = convolution_v3(im,H51)   # Zero-padded convolution (image in a "sea" of zeros).

plt.figure('FFT Convolution Artifacts',figsize=(12,12))
plt.subplot(221)
plt.imshow(out_f, cmap=plt.cm.gray,vmax=1)
plt.title("Convolution by fft (51x51)")
plt.subplot(222)
plt.imshow(out_direct, cmap=plt.cm.gray,vmax=1)
plt.title("Direct convolution (51x51)")
plt.subplot(223)
plt.imshow(np.abs(out_f - out_direct),cmap='jet',interpolation='nearest')
plt.title("Differences");
plt.colorbar(shrink=0.5);


In order to carry out the convolution properly and eliminate these effects, we have to pad the source image and the kernel with zeros.  It can be shown (see Gonzalez and Woods, "Digital Image Processing", 3rd ed.: library, and Brigham, "The Fast Fourier Transform": (available on the web),
that the needed padding is such as to make the padded images (both padded source and padded kernel) be of size $(R,C)$, where $R \geq R_i + R_H - 1$ and $C \geq C_i + C_H - 1$.  It is useful for efficiency to choose $R$ and $C$ even, and ideally, powers of 2.

The resulting image is larger than the source image, it needs to be cropped.  The cropping is explained in Brigham. (See also the workbook "Convolution and the FFT", which explains the padding and shift behaviour for 1-d signals, available in this directory).

Here is an implementation.

In [ ]:
import math

def convolution_fft_v2(im,H, verbose=False):
    """Perform convolution of an image by a kernel using the Fourier transform, this time
       with proper padding of both the source image and the kernel.
       
       Assumes the kernel is odd-sized and symmetric around the origin."""
    R_i, C_i = im.shape
    R_H, C_H = H.shape
    d_R, d_C = R_H // 2, C_H // 2
    R, C = (2**(np.ceil(np.log2([R_i + R_H - 1,C_i + C_H - 1])))).astype('int')
    if verbose:
        print(f"Image is {R_i} x {C_i}, Kernel is {R_H} x {C_H}.")
        print(f"Minimum padding is to {R_i + R_H - 1} x {C_i + C_H - 1}, "
              f"FFT on {R} x {C}.")
    dft_im = fft.rfft2(im, (R,C))
    dft_H = fft.rfft2(H, (R,C))
    return fft.irfft2(dft_im * dft_H)[d_R:d_R+R_i,d_C:d_C+C_i]  # Extract useful part of result.

In [ ]:
out_f_v2 = convolution_fft_v2(im, H51, verbose=True)

In [ ]:
plt.figure('FFT Convolution Artifacts',figsize=(12,12))
print("Padded FFT convolution: min and max values:", out_f_v2.min(),out_f_v2.max())
print("Direct, zero-padded convolution: min & max:",out_direct.min(), out_direct.max())
plt.subplot(221)
plt.imshow(out_f_v2, cmap=plt.cm.gray, vmax=1)
plt.title("Convolution by fft v2 (51x51)")
plt.subplot(222)
plt.imshow(out_direct, cmap=plt.cm.gray, vmax=1)
plt.title("Direct convolution (51x51)")
plt.subplot(223)
plt.imshow(np.abs(out_f_v2 - out_direct), cmap='jet',interpolation='nearest')
plt.title("Differences")
plt.colorbar(shrink=0.5);


The result of the FFT convolution looks like a direct convolution with zero padding.  There are minor differences between the FFT and the direct convolutions, but these are very small and due to numerical noise (note that the differences are all in the $10^{-8}$ region).

### FFT Speed

The direct convolution scales as $O(RCNM)$, whereas the FFT scales as $O(RC \log_2 R \log_2 C)$.  The cutoff where it can be beneficial to employ FFT convolution has to be determined by experiment, but generally, for large kernels like $51 \times 51$, it is an attractive option.  

In [ ]:
time_direct_numba_h51 = %timeit -o convolution_v3(im,H51)

In [ ]:
time_fft_h51 = %timeit -o convolution_fft_v2(im,H51)

In [ ]:
print(f"FFT Convolution speedup, 51x51 kernel: {time_direct_numba_h51.average/time_fft_h51.average:.2f}")

Here, for example, the FFT convolution is better than 15 times faster than direct convoution for the $51 \times 51$ kernel.

But it is slower for the $5 \times 5$ kernel.

In [ ]:
time_direct_numba_h5 = %timeit -o convolution_v3(im,H5)

In [ ]:
time_fft_h5 = %timeit -o convolution_fft_v2(im, H5)

In [ ]:
print(f"FFT Convolution speedup, 5x5 kernel: {time_direct_numba_h5.average/time_fft_h5.average:.2f}")

It is about twice as fast for the $11 \times 11$ kernel.  This illustrates that the FFT approach to convolution is attractive, albeit complicated.

In [ ]:
time_direct_numba_h11 = %timeit -o convolution_v3(im,H11)

In [ ]:
time_fft_h11 = %timeit -o convolution_fft_v2(im, H11)

In [ ]:
print(f"FFT Convolution speedup, 11x11 kernel: {time_direct_numba_h11.average/time_fft_h11.average:.2f}")

## Seperable Kernels

Another approach to speeding up convolution takes advantage of the fact that some kernels (not all) are separable.  This means that the 2-D convolution can be broken down into two 1-D convolutions.  The idea is that the overall 2-D convoution can be performed by first applying a 1-D convolution to the columns of the source image, then a second 1-D convolution to the rows of the 2-D array resulting from the first convolution.

In 2-D convolution the basic equation is:

\begin{equation}
                Y[u,v] = \sum_{i=-n}^{n} \sum_{j=-m}^{m} X[u-i,v-j] \; H[i,j]
\end{equation}


If $H$ is seperable, then $H[i,j] = H[i] H[j]$, so:

\begin{eqnarray*}
                Y[u,v] & = & \sum_{i=-n}^{n} \sum_{j=-m}^{m} X[u-i,v-j] \; H[i,j] \\
                       & = & \sum_{i=-n}^{n} \sum_{j=-m}^{m} X[u-i,v-j] \; H[i] \; H[j] \\
                       & = & \sum_{i=-n}^{n} \; \left( \sum_{j=-m}^{m} X[u-i,v-j] H[j] \right) \; H[i]
\end{eqnarray*}

The term inside the brackets is a 1-D convolution over columns (because, inside the $j$-sum, $i$ is constant).  Similarly, once the convolution within the brackets is performed, the result depends on $i$ only, so we have a second 1-D convolution over rows.

Here, for example, the $5 \times 5$ 2-d convolution kernel, which is seperable, is carried out by first performing 1-d convolutions of a 5-element kernel to the rows of the source image, then a second convolution columnwise to the result of this.  The overall effect is the same as performing a 2-d convolution with a $5 \times 5$ kernel.

In [ ]:
def convolution_sep_v1(im,Hr,Hc):
    """2-d convolution using a seperable kernel.  
    
              convolution_sep_v1(im,Hr,Hc)
              
       Parameter Hr must be the 1-d row component of an n x m seperable kernel, 
       and Hc the 1-d column component, such that outer(Hr,Hc) is the 2-D kernel.
    """
    R, C = im.shape
    N, M  = len(Hr), len(Hc)
    assert N % 2 == 1 and M % 2 == 1, "Dimensions of kernel must be odd lengths."
    n, m = N // 2, M // 2 
    #
    # Part (1), convolutions down rows, all columns in parallel.
    #
    temp=np.zeros(im.shape,dtype=im.dtype)  # Temporary image holds results of row convolutions.
    acc = np.zeros(C, dtype=im.dtype)       # Set of accumulators (1 per column) for row convolution calcs.
    for r in range(R):
        acc[:] = 0.0
        for i in range(N):
            u = r + n - i
            if u >= 0 and u < R: acc += Hr[i] * im[u,:]  # All columns accumulate in parallel.
        temp[r,:] = acc
    #
    # Part (2), convolutions across columns, all rows in parallel.
    #
    Y=np.zeros(im.shape,im.dtype)           # Y is output image
    acc = np.zeros(R, dtype=im.dtype)       # Set of accmulators (1 per row) for column convol calculations.
    for c in range(C):
        acc[:] = 0.0
        for j in range(M):
            v = c + m - j
            if v >= 0 and v < C: acc += Hc[j] * temp[:,v]  # All rows accumulate in parallel this time. 
        Y[:,c] = acc
    return Y

Here we make a seperable component of a square $5 \times 5$ kernel.  The point is that the outer product of this "Hs5" 1-D kernel with itself is the "H5" 2-D kernel.

In [ ]:
Hs5=np.ones(5)/5.0

In [ ]:
Hs5

Applying this kernel to the seperable convolution code give the same results as direct 2-D convolution by a $5 \times 5$ kernel (with some residual error due to the use of single-precision floats).

In [ ]:
Ys5=convolution_sep_v1(im,Hs5,Hs5)

In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(Ys5,cmap=plt.cm.gray,vmin=0,vmax=1)
plt.axis('off');

In [ ]:
imh5direct = convolution_v3(im,np.outer(Hs5,Hs5))

plt.figure(figsize=(6,6))
plt.imshow(imh5direct,cmap=plt.cm.gray,vmin=0,vmax=1)
plt.axis('off');

In [ ]:
plt.figure(figsize=(8,6))
plt.imshow(np.abs(Ys5-imh5direct),interpolation='nearest',cmap='hot')
plt.colorbar(shrink=0.5)
plt.axis('off');

In [ ]:
print(Ys5.mean(), Ys5.std(), Ys5.min(), Ys5.max())

In [ ]:
print(imh5direct.mean(), imh5direct.std(), imh5direct.min(), imh5direct.max())

Interestingly, even though the seperable convolution above is implemented as interpreted Python with NumPy, it is quite close to the speed of the compiled Cython direct 2-D convolution.  This is due partly to the use that the seperable convolution makes of the very efficient NumPy routines for array addition and multiplication on slices, but more importantly because the seperable convolution has a run time that scales as $O(RC(N+M))$ instead of $O(RCNM)$ for the direct convolution.

In [ ]:
time_sep_interp_h5 = %timeit -o convolution_sep_v1(im,Hs5,Hs5)

In [ ]:
time_direct_numba_h5 = %timeit -o convolution_v3(im,np.outer(Hs5,Hs5))

The effect is much more pronounced for a seperable version of the $51 \times 51$ square kernel used earlier.  Here the separable kernel implementation in interpreted Python is <em>faster</em> than the Numba-compiled direct convolution.

In [ ]:
Hs51=np.ones(51)/51.0

In [ ]:
%timeit convolution_sep_v1(im,Hs51,Hs51)

In [ ]:
%timeit convolution_v3(im,np.outer(Hs51,Hs51))

To speed things up futher, we could try converting the seperable convolution code to Numba. The only change we have to make is to add the @njit decorator.


In [ ]:
@njit
def convolution_sep_v2(im, Hr, Hc):
    """2-d convolution using a seperable kernel, compiled with Numba.  
    
              convolution_sep_v2(im,Hr,Hc)
              
       Parameter Hr must be the 1-d row component of an n x m seperable kernel, 
       and Hc the 1-d column component, such that outer(Hr,Hc) is the 2-D kernel.
       If Hc is omitted, it is assumed to be identical to Hr.
    """
    #if not Hc: Hc = Hr                      # If no Hc supplied, use a copy of Hr for the col convols.
    R, C = im.shape
    N, M  = len(Hr), len(Hc)
    assert N % 2 == 1 and M % 2 == 1, "Dimensions of kernel must be odd lengths."
    n, m = N // 2, M // 2 
    #
    # Part (1), convolutions down rows, all columns in parallel.
    #
    temp=np.zeros(im.shape,dtype=im.dtype)  # Temporary image holds results of row convolutions.
    acc = np.zeros(C, dtype=im.dtype)       # Set of accumulators (1 per column) for row convolution calcs.
    for r in range(R):
        acc[:] = 0.0
        for i in range(N):
            u = r + n - i
            if u >= 0 and u < R: acc += Hr[i] * im[u,:]  # All columns accumulate in parallel.
        temp[r,:] = acc
    #
    # Part (2), convolutions across columns, all rows in parallel.
    #
    Y=np.zeros(im.shape,im.dtype)           # Y is output image
    acc = np.zeros(R, dtype=im.dtype)       # Set of accmulators (1 per row) for column convol calculations.
    for c in range(C):
        acc[:] = 0.0
        for j in range(M):
            v = c + m - j
            if v >= 0 and v < C: acc += Hc[j] * temp[:,v]  # All rows accumulate in parallel this time. 
        Y[:,c] = acc
    return Y

In [ ]:
Ys5v2 = convolution_sep_v2(im, Hs5, Hs5)

In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(Ys5, cmap=plt.cm.gray)
plt.axis('off');

In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(Ys5v2, cmap=plt.cm.gray)
plt.axis('off');

In [ ]:
plt.figure(figsize=(8,6))
plt.imshow(np.abs(imh5direct-Ys5v2), cmap=plt.cm.hot)
plt.axis('off')
plt.colorbar(shrink=0.75);

The compiled Numba code working on the seperable version of the $5 \times 5$ kernel is about $5$ - $6$ times faster than its interpreted counterpart, and somewhat faster than the direct $5 \times 5$ kernel.  Of course, the differences become more pronounced for bigger images and bigger kernels.

The implementations of filters in the scipy.ndimage.filters package are aware of linear separabliity, and make use of it whenever possible (for the uniform_filter and for the Gaussian filter).

In [ ]:
time_sep_numba_h5 = %timeit -o convolution_sep_v2(im,Hs5,Hs5)

In [ ]:
time_sep_interp_h5.average / time_sep_numba_h5.average

In [ ]:
time_direct_numba_h5.average / time_sep_numba_h5.average

Of course, convolution by separation depends on the kernel being seperable, i.e., $H[i,j] = H[i]H[j]$.  This is not generally true.  For example, a "circular kernel", where

\begin{equation}
    H[i,j] = \left\{\begin{array}{cl}  1  &  (i^2+j^2) > N^2 \\ 0  & otherwise \end{array}\right.
\end{equation}

is not seperable.

However, the important Gaussian kernel

\begin{equation}
    H[i,j] = \exp(-(i^2+j^2)/2\sigma^2)
\end{equation}

is seperable, and provides a good appraoch to implementing a "circular-style" kernel where the importance of the contribution of a pixel diminishes exponentially quickly with its distance from the current convolution centre.

